<center><img src="../docs/assets/animated_logo.png" align="center" height="100" width="100">

# [`Optimus-Primal`](https://github.com/astro-informatics/Optimus-Primal) - __Basic 2D__ Interactive Tutorial
---

In this interactive tutorial we demonstrate basic usage of `optimusprimal` for a 2-dimensional image denoising problem.


How to run a basic 2D unconstrained proximal primal-dual solver. 
We consider the canonical problem $y = x + n$ where $n \sim \mathcal{N}$. 
This inverse problem can be solved via the unconstrained optimisation 

$$
\min_x [ ||(x-y)/\sigma||^2_2 + \lambda ||\Psi^{\dagger} x||_1 ]
$$

where $x \in \mathbb{R}^2$ is an a priori ground truth 2D signal, $y \in \mathbb{R}^2$ 
are simulated noisy observations, and $\lambda$ is the regularisation parameter which acts as 
a Lagrangian multiplier, balancing between data-fidelity and prior information. Before we begin, we 
need to import ``optimusprimal`` and some example specific packages


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc as misc 
from scipy.signal import resample

import optimusprimal.primal_dual as primal_dual
import optimusprimal.grad_operators as grad_operators
import optimusprimal.linear_operators as linear_operators
import optimusprimal.prox_operators as prox_operators

First, we need to define some heuristics for the solver, these include:

      - tol: convergence criteria for the iterations
      - iter: maximum number of iterations
      - update_iter: iterations between logging iteration diagnostics
      - record_iters: whether to record the full diagnostic information



In [4]:
options = {"tol": 1e-5, "iter": 5000, "update_iter": 50, "record_iters": False}

Next, we simulate a standard de-noising setting by contaminating a known
signal $x$ with some Gaussianly distributed noise. Note that for simplicity the
measurement operator here is taken to be the identity operator.



In [5]:
ISNR = 25.0                                            # Input signal to noise ratio
sigma = 10 ** (-ISNR / 20.0)                           # Noise standard deviation
reg_param = 2.5                                        # Regularisation parameter
res = 256                                              # Resolution we want to work with

x = misc.ascent()                                      # Scipy's ascent benchmark image
for i in range(2):
    x = resample(x, axis=i, num=res)
x /= np.nanmax(x)                                      # Normalise image

n = np.random.normal(0, sigma, x.shape)                # Random Gaussian noise
y = x + n                                              # Simulated observations y

/var/folders/zc/g9pg19x15rg0rz8wcsr79645xjx6sw/T/ipykernel_12931/2168788683.py:6: DeprecationWarning: scipy.misc.ascent has been deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. Dataset methods have moved into the scipy.datasets module. Use scipy.datasets.ascent instead.
  x = misc.ascent()                                      # Scipy's ascent benchmark image


For the unconstrained problem with Gaussian noise the data-fidelity constraint
is given by the gradient of the $\ell_2$-norm. Here we set up a gradient operator
corresponding to a gradient of the $\ell_2$-norm.



In [ ]:
g = grad_operators.l2_norm(sigma, y, linear_operators.identity())
g.beta = 1.0 / sigma ** 2

We regularise this inverse problem by adopting a wavelet sparsity $\ell_1$-norm prior.
To do this we first define what wavelets we wish to use, in this case a
combination of Daubechies family wavelets, and which levels to consider.
Any combination of wavelet families available by the [`PyWavelet`](https://tinyurl.com/5n7wzpmb) package may be
selected.



In [ ]:
wav = ["db1", "db4"]                                     # Wavelet dictionaries to combine
levels = 4                                               # Wavelet levels to consider [1-6]
psi = linear_operators.dictionary(wav, levels, x.shape)  # Wavelet linear operator

Next we construct the $\ell_1$-norm proximal operator which we pass the wavelets
($\Psi$) as a dictionary in which to compute the $\ell_1$-norm. We also add an
additional reality constraint f for good measure, as we know a priori our
signal $x$ is real.



In [ ]:
h = prox_operators.l1_norm(np.max(np.abs(psi.dir_op(y))) * reg_param, psi)
h.beta = 1.0
f = prox_operators.real_prox()

Finally we run the optimisation...



In [ ]:
best_estimate, diagnostics = primal_dual.FBPD(y, options, g, f, h)

...and plot the results!



In [ ]:
def eval_snr(x, x_est):
    if np.array_equal(x, x_est):
        return 0
    num = np.sqrt(np.sum(np.abs(x) ** 2))
    den = np.sqrt(np.sum(np.abs(x - x_est) ** 2))
    return round(20*np.log10(num/den), 2)

fig, axs = plt.subplots(1, 3, figsize=[10, 5])

titles = ["Data", "Truth", "Reconstruction"]
est = [y, x, best_estimate]

for i in range(3):
    axs[i].imshow(est[i], cmap="magma", vmax=np.max(x), vmin=np.min(x))
    axs[i].set_title(titles[i], fontsize=14)
    axs[i].set_xlabel("SNR: {}dB,".format(eval_snr(x, est[i])), fontsize=12)

    plt.setp(axs[i].get_xticklabels(), visible=False)
    plt.setp(axs[i].get_yticklabels(), visible=False)

plt.show()